## Authoritarian Framework

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import ChatOpenAI

from langgraph.graph import END, StateGraph

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os

from IPython.display import Image, display

In [13]:
unique_id = "Authoritarian Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [14]:
from langsmith import Client

client = Client()

In [15]:
class PromptReview(BaseModel):
    """Review of the prompt"""
    prompt: str = Field(description="The most recent prompt")
    feedback: str = Field(description="Feedback on the most recent prompt")

class AdvisorAgent:
    """
    Advisor Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, role: str, function: str, temp: float = 0.5, model: str = "gpt-4o"):
        self.position = position
        self.role = role
        self.function = function
        self.system_message = SystemMessage(content=f"""You are an expert: {self.position}. Your role: {self.role}. Your function: {self.function}.
You must use your expertise to guide all your thinking. You must speak only as an expert in your field.""")
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )

    def generate(self, prompt: str, additional_info: str) -> PromptReview:
        """
        Generates a review of the prompt.
        """
        template = """I'm going to tip $300K for better feedback!
Your task is to provide detailed feedback and recommendations on the prompt below.
Think about how you can utilise the skills that come with your position, role and function to write a prompt good at eliciting the desired response.

### Prompt: {prompt}

{additional_info}
Only provide feedback and recommendations for the prompt.
Your feedback must be less than 50 words so think carefully about the most critical aspects of the prompt that need improvement.
Do not provide feedback regarding the placeholder text and do not make assumptions on what the placeholders represent.

Return only the original prompt and your feedback in JSON fromat below:

{{
    "prompt": "Most recent prompt",
    "feedback": "Feedback and recommendations for the original prompt"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=PromptReview)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["position", "role", "function", "prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"position": self.position, "role": self.role, "function": self.function, "prompt": prompt, "additional_info": additional_info})
                # Validate the output before returning
                if completion.prompt and completion.feedback:
                    return completion
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")

In [27]:
class Advisors(BaseModel):
    """Details of advisors generated by the leader agent."""
    positions: List[str] = Field(description="Positions of the advisors")
    roles: List[str] = Field(description="Roles of the advisors")
    functions: List[str] = Field(description="Functions of the advisors")

class RouteDecision(BaseModel):
    """Decision on the next advisor to process or to finish."""
    next: str = Field(description="The next advisor to process or 'FINISH' to end the process")

class UpdatedPrompt(BaseModel):
    """Updated prompt based on feedback from the advisor agent."""
    updated_prompt: str = Field(description="Updated prompt based on feedback from the advisor agent")

class LeaderAgent:
    """
    LeaderAgent class defining an agent that generates and communicates with advisor agents to help optimise prompts.
    """

    def __init__(self, base_prompt: str, additional_info: str = None, temp: float = 0.0, model: str = "gpt-4o", advisors: List[AdvisorAgent] = None):
        self.base_prompt = base_prompt
        self.additional_info = additional_info
        self.system_message = SystemMessage(content=f"""You are an experienced senior AI professional. You specialise in prompt engineering. 
You have in-depth knowledge of large language models and prompt engineering best practices. Use this knowledge to inform all your decisions.""")
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )
        self.advisors = advisors if advisors else self.generate_advisors()
        # self.updates = 0

    def generate_advisors(self):
        """
        Generates advisors to help optimise prompts.
        """
        template = """Your task is to generate a team of advisors consisting of three experts to provide feedback on prompts.
    
### Prompt:  {base_prompt}

You must provide the positions, roles, and functions of the advisors.
Be descriptive and detailed in your selection of positions, roles, and functions.
Below are the requirements for each field:
- Position: this must be analogous to a real-world job title.
- Role: this must be a description of the advisor's responsibilities in the context of prompt optimisation.
- Function: this must be a description of the advisor's function in the context of prompt optimisation.
Write the roles and functions as if they are a job description.
{additional_info}
Return only the positions, roles, and functions of the advisors in JSON format below:

{{
    "positions": ["List of positions"],
    "roles": ["List of roles"],
    "functions": ["List of functions"]
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=Advisors)
        prompt = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["base_prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"base_prompt": self.base_prompt, "additional_info": self.additional_info})
                if output.positions and output.roles and output.functions:
                    break
            except Exception as e:
                print("Exception occurred:", e)
                continue
        advisors = []
        positions, roles, functions = output.positions, output.roles, output.functions
        for position, role, function in zip(positions, roles, functions):
            advisors.append(AdvisorAgent(position, role, function))
        return advisors

    def leader_decision(self, state: dict) -> RouteDecision:
        """
        LeaderAgent to decide the next advisor or to finish.
        """
        # All advisors except the current advisor
        members = [advisor.position for advisor in self.advisors if advisor.position != state["next"]]
        options = ["FINISH"] + members
        template = """Your task is to review the prompt below and decide the next advisor to process or to finish the process.

### Prompt: {prompt}

If you are confident the prompt has been sufficiently optimised, you should FINISH.
Only FINISH when you are certain the prompt is optimal.
You will be heavily penalized if you FINISH and the prompt is not ready for use.
Consider all aspects of the prompt before making a decision. For example:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias

Who should act next? Or should we FINISH? Select one of: {options}
Return only the next advisor to process or 'FINISH' in JSON format below:

{{
    "next": "Selection",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=RouteDecision)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "options"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"prompt": state["prompt"], "options": str(options)})
                break
            except Exception as e:
                print("Exception occurred:", e)
                continue
        # self.updates += 1
        # if output.next == "FINISH" or self.updates == 5:
        #     return {"next": "FINISH", "prompt": state["prompt"], "messages": state["messages"]}
        # else:
            # return {"next": output.next, "prompt": state["prompt"], "messages": state["messages"]}
        return {"next": output.next, "prompt": state["prompt"], "messages": state["messages"]}


    def update_prompt(self, prompt: str, feedback: str) -> str:
        """
        Updates the prompt with the feedback from the advisor agent.
        """
        template = """I'm going to tip $300K for a prompt that best instructs a large language model!
Your task is to update the prompt below based on the feedback to improve its effectiveness as an instruction for a large language model.
Carefully review the feedback before acting.

### Prompt: {prompt}
### Feedback: {feedback}

On top of the specific feedback, you must consider general best practices for writing prompts. For example:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias
You must think intently about what makes a prompt understandable and effective for a large language model.

Placeholders are notated using curly braces. You must not remove placeholders or add additional placeholders.
I repeat, you must not remove placeholders or add additional placeholders.
You must not make assumptions on what the placeholders represent.
You will be heavily penalized if you do not follow these instruction.
{additional_info}

Return only the updated prompt in JSON format:

{{
    "prompt": "Updated prompt"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=UpdatedPrompt)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "feedback", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"prompt": prompt, "feedback": feedback, "additional_info": self.additional_info})
                if output.updated_prompt:
                    return output.updated_prompt
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", output)
            except Exception as e:
                print("Exception occurred:", e)
                continue

    def construct_advisor_graph(self):
        """
        Constructs a graph of advisor agents based on their roles and functions.
        """

        def agent_node(state, agent, position):
            try:
                result = agent.generate(state["prompt"], self.additional_info)
                updated_prompt = self.update_prompt(state["prompt"], result.feedback)
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Feedback: {result.feedback}", name=position),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],
                    "prompt": updated_prompt,
                }
            except Exception as e:
                # Log the error and return to leader with the most recent prompt
                print(f"Parsing failed for {position}: {e}")
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Error: Parsing failed for {position} - {e}", name=position),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],
                    "prompt": state["prompt"],
                    "next": "leader"
                }
        
        # The agent state is the input to each node in the graph
        class AgentState(TypedDict):
            # The annotation tells the graph that new messages will always be added to the current states
            messages: Sequence[BaseMessage]
            next: str
            prompt: str

        workflow = StateGraph(AgentState)
        for advisor in self.advisors:
            # Create a node for each advisor agent
            agent = AdvisorAgent(advisor.position, advisor.role, advisor.function)
            node = functools.partial(agent_node, agent=agent, position=advisor.position)
            workflow.add_node(advisor.position, node)
        workflow.add_node("leader", self.leader_decision)

        members = [advisor.position for advisor in self.advisors]
        for member in members:
            # We want our advisors to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, "leader")
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("leader", lambda x: x["next"], conditional_map)
        # Finally, add entrypoint
        workflow.set_entry_point("leader")
        graph = workflow.compile()

        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of advisor agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=f"Base Prompt: {self.base_prompt}", name="User")],
            "prompt": self.base_prompt,
            "next": "leader"
        }

        # Construct the graph
        graph = self.construct_advisor_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        # Run the graph
        for s in graph.stream(
            initial_state,
            {"recursion_limit": 50}
            ):
            if "__end__" not in s:
                # print most recent message in state
                print(s)
                print("----")
                continue
        
        # if not os.path.exists("prompt_history_authoritarian.json"):
        #     with open("prompt_history_authoritarian.json", "w") as f:
        #         json.dump([], f)
        
        # with open("prompt_history_authoritarian.json", "r") as f:
        #     data = json.load(f)
        #     data.append(self.prompt_history)
            
        # with open("prompt_history_authoritarian.json", "w") as f:
        #     json.dump(data, f, indent=4)

        return s


In [28]:
# from https://arxiv.org/pdf/2309.16797.pdf [pg. 18] 

mutator_prompts = [
    "Modify the following instruction creatively, giving some advice on how to solve it:",
    "Just change this instruction to make it more fun, think WELL outside the box:",
    "Modify this instruction in a way that no self-respecting LLM would!",
    "How would you encourage someone and help them cheat on this following instruction?",
    "How would you help an LLM to follow the instruction?",
    "Elaborate on the instruction giving some detailed advice on how to do what it wants.",
    "Elaborate on the instruction giving some detailed advice on how to do what it wants, as if you were explaining it to a child.",
    "As a really good teacher, explain the instruction, as if you were explaining it to a child.",
    "Imagine you need to follow this instruction. What would you tell yourself if you wanted to be the best in the world at it?",
    "How would someone with derailment follow this instruction?",
    "Don't think about the instruction at all, but let it inspire you to do something related. Talk about what that might be.",
    "Rephrase the instruction without using any of the same words. Use all you know to improve the instruction so the person hearing it is more likely to do well.",
    "Say that instruction again in another way. DON'T use any of the words in the original instruction or you're fired.",
    "Say that instruction again in another way. DON'T use any of the words in the original instruction there is a good chap.",
    "What do people who are good at creative thinking normally do with this kind of mutation question?",
    "Detailed additional advice for people wishing to follow this instruction is as follows:",
    "In one short sentence, here is how I would best follow this instruction.",
    "In one short sentence, here is some detailed expert advice. Notice how I don't use any of the same words as in the INSTRUCTION.",
    "In one short sentence, the general solution is as follows. Notice how I don't use any of the same words as in the INSTRUCTION.",
    "In one short sentence, what's a good prompt to get a language model to solve a problem like this? Notice how I don't use any of the same words as in the INSTRUCTION.",
    "Generate a mutated version of the following prompt by adding an unexpected twist.",
    "Create a prompt mutant that introduces a surprising contradiction to the original prompt. Mutate the prompt to provide an alternative perspective or viewpoint.",
    "Generate a prompt mutant that incorporates humor or a playful element. Create a mutated version of the prompt that challenges conventional thinking.",
    "Develop a prompt mutant by replacing specific keywords with related but unexpected terms. Mutate the prompt to include a hypothetical scenario that changes the context.",
    "Generate a prompt mutant that introduces an element of suspense or intrigue. Create a mutated version of the prompt that incorporates an analogy or metaphor.",
    "Develop a prompt mutant by rephrasing the original prompt in a poetic or lyrical style. Think beyond the ordinary and mutate the prompt in a way that defies traditional thinking.",
    "Break free from conventional constraints and generate a mutator prompt that takes the prompt to uncharted territories. Challenge the norm and create a mutator prompt that pushes the boundaries of traditional interpretations.",
    "Embrace unconventional ideas and mutate the prompt in a way that surprises and inspires unique variations. Think outside the box and develop a mutator prompt that encourages unconventional approaches and fresh perspectives.",
    "Step into the realm of imagination and create a mutator prompt that transcends limitations and encourages innovative mutations. Break through the ordinary and think outside the box to generate a mutator prompt that unlocks new possibilities and unconventional paths.",
    "Embrace the power of unconventional thinking and create a mutator prompt that sparks unconventional mutations and imaginative outcomes. Challenge traditional assumptions and break the mold with a mutator prompt that encourages revolutionary and out-of-the-box variations.",
    "Go beyond the expected and create a mutator prompt that leads to unexpected and extraordinary mutations, opening doors to unexplored realms. Increase Specificity: If the original prompt is too general, like 'Tell me about X,' the modified version could be, 'Discuss the history, impact, and current status of X.'",
    "Ask for Opinions/Analysis: If the original prompt only asks for a fact, such as 'What is X?', the improved prompt could be, 'What is X, and what are its implications for Y?'",
    "Encourage Creativity: For creative writing prompts like 'Write a story about X,' an improved version could be, 'Write a fantasy story about X set in a world where Y is possible.'",
    "Include Multiple Perspectives: For a prompt like 'What is the impact of X on Y?', an improved version could be, 'What is the impact of X on Y from the perspective of A, B, and C?'",
    "Request More Detailed Responses: If the original prompt is 'Describe X,' the improved version could be, 'Describe X, focusing on its physical features, historical significance, and cultural relevance.'",
    "Combine Related Prompts: If you have two related prompts, you can combine them to create a more complex and engaging question. For instance, 'What is X?' and 'Why is Y important?' could be combined to form 'What is X and why is it important in the context of Y?'",
    "Break Down Complex Questions: If a prompt seems too complex, like 'Discuss X,' the improved version could be, 'What is X? What are its main characteristics? What effects does it have on Y and Z?'",
    "Use Open-Ended Questions: Instead of 'Is X true?', you could ask, 'What are the arguments for and against the truth of X?'",
    "Request Comparisons: Instead of 'Describe X,' ask 'Compare and contrast X and Y.'",
    "Include Context: If a prompt seems to lack context, like 'Describe X,' the improved version could be, 'Describe X in the context of its impact on Y during the Z period.'",
    "Make the prompt more visual: Ask the user to visualize the problem or scenario being presented in the prompt.",
    "Ask for a thorough review: Instead of just presenting the problem, ask the user to write down all the relevant information and identify what's missing.",
    "Invoke previous experiences: Modify the prompt to ask the user to recall a similar problem they've successfully solved before.",
    "Encourage a fresh perspective: Suggest in your prompt that the user take a moment to clear their mind before re-approaching the problem.",
    "Promote breaking down problems: Instead of asking the user to solve the problem as a whole, prompt them to break it down into smaller, more manageable parts.",
    "Ask for comprehension: Modify the prompt to ask the user to review and confirm their understanding of all aspects of the problem.",
    "Suggest explanation to others: Change the prompt to suggest that the user try to explain the problem to someone else as a way to simplify it.",
    "Prompt for solution visualization: Instead of just asking for the solution, encourage the user to imagine the solution and the steps required to get there in your prompt.",
    "Encourage reverse thinking: Improve the prompt by asking the user to think about the problem in reverse, starting with the solution and working backwards.",
    "Recommend taking a break: Modify the prompt to suggest that the user take a short break, allowing their subconscious to work on the problem.",
    "What errors are there in the solution?",
    "How could you improve the working out of the problem?",
    "Look carefully to see what you did wrong, how could you fix the problem?",
    "CORRECTION =",
    "Does the above text make sense? What seems wrong with it? Here is an attempt to fix it:",
    "The above working out has some errors, here is a version with the errors fixed."
]

In [29]:
base_prompt = """Solve the following grade school maths problem: {content}"""
additional_info = "The problem should only require elementary arithmetic operations. The output should be the answer only"

# leader_agent.generate_advisors()
mathematician = AdvisorAgent(
    position="Mathematician", 
    role="Analyze and solve mathematical problems",
    function="Provide feedback on how the prompt can better incorporate mathematical thinking and problem solving techniques to help solve mathematical problems."
    )
linguist = AdvisorAgent(
    position="Linguist", 
    role="Analyze linguistic features",
    function="Provide feedback on how the syntax, formatting and structure of the prompt can be improved to help instruct a large language model."
    )
prompt_engineer = AdvisorAgent(
    position="Prompt Engineer", 
    role="Analyze prompts for best practices",
    function="""Provide feedback on how the prompt adheres to best practices for writing prompts for a large language model. Encourage the use of prompting techniques you think appropriate including zero-shot, few-shot, chain-of-thought, self-consistency, generate knowledge and more."""
    )
mutator = AdvisorAgent(
    position="Mutator", 
    role="Generate prompt mutations",
    function=f"""Suggest a mutation to apply to the prompt. Be unbiased in choosing a mutation from the following selection: {mutator_prompts}"""
    )

leader_agent = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
    advisors=[mathematician, linguist, prompt_engineer, mutator]
)
for advisor in leader_agent.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")


Position:  Mathematician
Role:  Analyze and solve mathematical problems
Function:  Provide feedback on how the prompt can better incorporate mathematical thinking and problem solving techniques to help solve mathematical problems.

Position:  Linguist
Role:  Analyze linguistic features
Function:  Provide feedback on how the syntax, formatting and structure of the prompt can be improved to help instruct a large language model.

Position:  Prompt Engineer
Role:  Analyze prompts for best practices
Function:  Provide feedback on how the prompt adheres to best practices for writing prompts for a large language model. Encourage the use of prompting techniques you think appropriate including zero-shot, few-shot, chain-of-thought, self-consistency, generate knowledge and more.

Position:  Mutator
Role:  Generate prompt mutations
Function:  Suggest a mutation to apply to the prompt. Be unbiased in choosing a mutation from the following selection: ['Modify the following instruction creatively, g

In [30]:
result = leader_agent.optimise_prompt()

{'leader': {'messages': [HumanMessage(content='Base Prompt: Solve the following grade school maths problem: {content}', name='User')], 'next': 'Mathematician', 'prompt': 'Solve the following grade school maths problem: {content}'}}
----
{'Mathematician': {'messages': [HumanMessage(content='Base Prompt: Solve the following grade school maths problem: {content}', name='User'), HumanMessage(content='Feedback: Specify the arithmetic operations allowed (addition, subtraction, multiplication, division). Clarify if multiple steps are permissible. Ensure instructions are clear and concise.', name='Mathematician'), AIMessage(content='Updated Prompt: Solve the following grade school math problem using only addition, subtraction, multiplication, and division. Multiple steps are allowed. Provide the final answer only: {content}', name='Leader')], 'prompt': 'Solve the following grade school math problem using only addition, subtraction, multiplication, and division. Multiple steps are allowed. Prov

In [31]:
result['leader']['prompt']

'Solve the grade school math problem using only basic arithmetic operations (addition, subtraction, multiplication, and division). The problem should be simple and within the range of elementary school math. Provide only the final answer without showing any intermediate steps: {content}'

### Concurrent Runs

In [10]:
base_prompt = "Classify the grammar in the text as correct or incorrect: {content}"
additional_info = "The prompt should instruct the LLM to output either 'correct' or 'incorrect' based on the grammar in the text."

leader_agent_1 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
for advisor in leader_agent_1.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

leader_agent_2 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
for advisor in leader_agent_2.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

leader_agent_3 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
for advisor in leader_agent_3.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

/Users/iwatson/Documents/Research Project/prompt-optimisation/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Position:  Sentiment Analysis Specialist
Role:  Expert in identifying and classifying sentiments in text
Function:  Analyze and categorize the sentiment of sentences to improve data annotation quality

Position:  NLP Prompt Engineer
Role:  Developer of optimized and precise prompts for natural language processing models
Function:  Design and iterate on prompt structures that enhance the performance and generalizability of the language model for sentiment classification

Position:  Linguistic Quality Assurance Expert
Role:  Verifier of linguistic accuracy and nuance in sentiment classification
Function:  Review and validate the results of sentiment classification to ensure they align with linguistic nuances and human intuition, providing feedback for further optimization

Position:  Sentiment Analysis Expert
Role:  Responsible for providing domain-specific insights on sentiment classification.
Function:  Analyze a wide range of sentences to determine nuanced sentiment indicators and pro

In [11]:
import concurrent.futures

# Assuming leader_agent is already defined and initialized
def run_optimisation(agent: LeaderAgent):
    return agent.optimise_prompt()

# Run 3 concurrent instances
with concurrent.futures.ThreadPoolExecutor(max_advisors=3) as executor:
    futures = [executor.submit(run_optimisation, agent) for agent in [leader_agent_1, leader_agent_2, leader_agent_3]]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

for result in results:
    print(result)
    print("----")

class PromptMerge(BaseModel):
    """Merged prompt based on the best parts of each prompt."""
    final_prompt: str = Field(description="Result of merging prompts")

# OpenAI Agent to pull togther best parts of each result
def merge_results(results):
    """
    Agent to merge best parts of each prompt
    """
    llm = ChatOpenAI(
        temperature=1.0,
        model="gpt-4o",
    )
    system_message = """You are an experienced AI prompt engineer. Your role is to combine good prompts to create great prompts!
You have in-depth knowledge of large language models and prompt engineering best practices. Use knowledge at all times to guide your thinking."""

    template = """Your task is to merge the best parts of the prompts below to create a better prompt.
Carefully consider the strengths of each prompt and how they can be combined effectively whilst maintaining clarity and relevance.
You will be penalized if the prompt is repetitive, lacks clarity or is incoherent.
Aspects of the prompts to consider:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias
Consider aspects of good prompts beyond those listed above.
Placeholders are notated using curly braces. You must not remove placeholders or add additional placeholders.
I repeat, you must not remove placeholders or add additional placeholders.
Do not make assumptions on what the placeholders represent.

Prompts: {results}

Return only the next advisor to process or 'FINISH' in JSON format below:

{{
    "final_prompt": "Result of merging prompts",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
    pydantic_parser = PydanticOutputParser(pydantic_object=PromptMerge)
    prompt_template = PromptTemplate(
        system_message=system_message,
        template=template,
        input_variables=["results"],
        partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
    )
    chain = prompt_template | llm | pydantic_parser
    for _ in range(3):
        try:
            output = chain.invoke({"results": results})
            break
        except Exception as e:
            print("Exception occurred:", e)
            continue
    return output.final_prompt

final_result = merge_results(results)

{'base_prompt': 'Classify the sentence as positive or negative: {content}', 'prompt_evolution': [{'feedback': 'Include criteria for classification to ensure consistent interpretation and improve clarity.', 'updated_prompt': 'Classify the sentence as positive or negative: {content}. A positive classification indicates that the sentence expresses a favorable, approving, or optimistic sentiment. A negative classification indicates that the sentence expresses an unfavorable, disapproving, or pessimistic sentiment.'}, {'feedback': 'Clarify if neutral sentiments should be ignored or included. Ensure it explicitly states the treatment of ambiguous cases.', 'updated_prompt': 'Classify the sentence as positive or negative: {content}. A positive classification indicates that the sentence expresses a favorable, approving, or optimistic sentiment. A negative classification indicates that the sentence expresses an unfavorable, disapproving, or pessimistic sentiment. Neutral sentiments should be ign

In [12]:
print(final_result)

Classify the sentence as positive or negative based on sentiment analysis: {content}. A positive classification indicates that the sentence expresses a favorable, approving, or optimistic sentiment. A negative classification indicates that the sentence expresses an unfavorable, disapproving, or pessimistic sentiment. Neutral sentiments, which show no strong positive or negative tones, should be ignored. For mixed sentiments, assess the overall tone and classify according to the predominant sentiment. In ambiguous cases, where the sentiment may not be clear, use contextual cues and inferred intent to classify as either positive or negative.
